In [17]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import path
from pathlib import Path
import shutil
import gdown
from exif import Image

# Project scope

**Mi camino** aims to be a little project for tracking my daily progress during my *Camino de Santiago* on bike. It will consist of several Python scripts that perform the following actions:

- Download all files in a *Google Drive* folder (gpx files and pictures) at set intervals.
- Parse the gpx files.
- Otain the location of the pictures using the metadata.
- Display the latest progress (parsed gpx files) on a map via *Folium*.
- Send the pictures to my *Raspberry* and display them on the map with a marker on the point they were taken.
- Display the current progress (km/elevation).

For my convenience I chose to use a *Google Drive* folder to store all gpx files and pictures that I want to upload during the route. This way I can limit the data uploads to once/twice a day and I can keep the scripts *relatively* simple, with just a few lines of web scraping.

## Downloading data from *Google Drive*

*Google* provides an API to interact with *Drive*, but since I only want to download a few files at a time and perform no uploads whatsoever I found a much simpler way to do so via the *gdown* library.

Once you have downloaded the folder it's simply a matter of accessing it using our beloved *Selenium*. Please notice that I'm not using the regular *chromedriver* but a library that automatically downloads and runs it for you, eliminating the risk of an out-of-date chromedriver.

In [6]:
# For security purposes it's good practice to store private links in txt files and add the to the gitignore

link = open('download_link.txt','r').readline() # Reading the file containing the link

In [8]:
# Downloading the folder using gdown

url = link
gdown.download_folder(url, quiet=True, use_cookies=False)

['C:\\Users\\User\\micamino\\camino\\Bicicleta_a_la_hora_del_almuerzo.gpx',
 'C:\\Users\\User\\micamino\\camino\\PXL_20220629_145528615.jpg']

## Unzipping the file and reading its contents

The driver now contains both types of files that we'll encounter, *gpx* files and pictures. Let's access them.

In [9]:
directory = r'C:\Users\User\micamino\camino' # Out download folder
files = Path(directory).glob('*') # Using all files in the folder as input
files = list(files)

gpx = [] # We'll hold gpx files here
images = [] # Same for images
for file in files:
    if '.jpg' in str(file): # Filtering by filename
        images.append(file)
    elif '.gpx' in str(file):
        gpx.append(file)
    else:
        pass        

In [10]:
gpx

[WindowsPath('C:/Users/User/micamino/camino/Bicicleta_a_la_hora_del_almuerzo.gpx')]

In [11]:
images

[WindowsPath('C:/Users/User/micamino/camino/PXL_20220629_145528615.jpg')]

In [19]:
# Let's open the image

img_path = images[0] # Using the path we just obtained
with open(img_path, 'rb') as src:
    img = Image(src)

In [20]:
# Now let's access its longitude

img.gps_longitude

(2.0, 24.0, 8.57)

As we can see, the longitude and latitude are in degrees, minutes and seconds. We'll need to use a little function to convert those coordinates to decimal degrees, as well as making the process more streamlined.

In [28]:
# Let's first define a function that simply converts the coordinates to decimal degrees.
# We'll have to take into account the orientation (ref), because the result will vary whether it's facing south or west.

def converter(coords, ref):
    #####
    #Input: coordinates and ref (orientation) of the picture, as expressed by the parser
    # Output: if there's coordinates, returns them in decimal degrees.
    #####
    decimal_degrees = coords[0] + coords[1] / 60 + coords[2] / 3600 # Converting to decimal degrees
    if ref == 'S' or ref == 'W':
        decimal_degrees = -decimal_degrees # Changing sign if it's facing south or west
    return decimal_degrees

In [30]:
def coordinates(image_path):
    with open(img_path, 'rb') as src:
        img = Image(src)    
    if img.has_exif:
        try:
            img.gps_longitude
            coords = (converter(img.gps_latitude,
                      img.gps_latitude_ref),
                      converter(img.gps_longitude,
                      img.gps_longitude_ref))
        except:
            return False
    else:
        return False
        
    return coords

In [33]:
coordinates(images[0])

(41.65405277777778, 2.4023805555555553)